# Model typical use case
## Imports

In [ ]:
%matplotlib inline
from IPython.display import display
import numpy as np
from matplotlib import pyplot as plt
import os
from useful_functions import *
import cPickle as pkl
import pandas as pd
from datetime import date
import ClassModel
InteractionModel = ClassModel.InteractionModel

## Definition of the useful paths, the manual variables and loading of the necessary data

- `stages` defines the name of the stages that will be use by the model and their order (Stage 9 is omitted on purpose).
- `cells_to_find_induction` are the cell that are known to undergo specification events
- `cells in grey` are the cell that are manually added to the grey zone 

- `basic_kinetic_group` is the default kinetic behaviour
- `kinetic_groups` is the dictionary that maps a ligand group to its kinetic behavior. The number represent the number of stages before being secreted at the cell interface.
- `small_half_life` is the list of the ligands groups that have a small half life (ie 1 stage) if they are not in this list they are considered to have a long half life (ie 2 stages)
- `z_c_end` is the daughter's maximum zygotic cell cycle acceptable by the model.
- `time_end` is the latest time point to consider for the model
- `begin` is the offset after division to consider the surface of contact (waiting for the division to complete)
- `end` is the lenght of the time-window considered for the surface of contact
- `dt` is the delta of time used for the numerical integration (it does not impact the result when the Euler method is not enforced)

Both `z_c_end` AND `time_end` have to be respected for a cell division event to be taken into account

`load_model_data` loads all the data necessary for the model to compute

parameter_sweep_output.pkl file contains the outputs of the parameter sweep performed for this particular data set

In [ ]:
path_to_data = 'DATA/'

stages = np.array(['Stage 5', 'Stage 6', 'Stage 8', 'Stage 10', 'Stage 11', 'Stage 12', 'Stage 13'])
cells_to_find_induction = ['a6.2', 'a6.3', 'a6.4', 'a6.7', 'b6.2', 'b6.4',
                           'a7.8', 'a7.9', 'a7.10', 'a7.13', 'b7.3',
                           'a8.7', 'a8.8', 'a8.17', 'a8.19', 'a8.25',
                           'a8.15', 'a8.16']
cells_in_grey = ['a7.4', 'a6.1', 'b6.1', 'b7.2', 'b8.19']

basic_kinetic_group = 1
kinetic_groups={
    'Nodal+':0,
    'Nodal-':0,
    'Notch+':0,
    'Notch-':0,
    'BMP-':0,
    'BMP+':0
}
small_half_life = ['Nodal+', 'Nodal-','Notch+','Notch-','BMP-','BMP+', 'Wnt+', 'Wnt-', 'ERK+', 'ERK-']

z_c_end = 9
time_end = 180

begin = 2
end = 100
dt = 2

(pos_AP, lin_tree, fates,
fates2, fates3, vol, sim_nv,
inv_lin_tree, surf_ex, names,
surfaces, bary, properties,
ColorMap, gene_expression,
ground_truth, ground_truth_pw,
fates3_names, couples, known_interactions,
gene_affect_neighb, gene_affect_self, 
pathways, gene_effect) = load_model_data(path_to_data, stages, kinetic_groups,
                                         basic_kinetic_group, small_half_life,
                                         z_c_end, time_end)
output_folder = 'Model-Output-'+date.today().isoformat()+'/'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

## Loading of the parameter sweep results

In [ ]:
f = open('DATA/param-sweep-version-5.1.1.pkl')
FP_all, FN_all, MISSED_L_all, COORD_all = pkl.load(f)
f.close()

f = open('DATA/param-sweep-version-5.1.2.pkl')
FP_all_tmp, FN_all_tmp, MISSED_L_all_tmp, COORD_all_tmp = pkl.load(f)
f.close()

FP_all = list(FP_all) + list(FP_all_tmp)
FN_all = list(FN_all) + list(FN_all_tmp)
MISSED_L_all = list(MISSED_L_all) + list(MISSED_L_all_tmp)
COORD_all.update(COORD_all_tmp)


output_folder = 'Model-Output-'+date.today().isoformat()+'/'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
R = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,0]))}
T = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,1]))}
I_T = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,3]))}
KF = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,4]))}
D6 = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,6]))}
D8 = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,7]))}
D10 = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,8]))}
D1 = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,5]))}
ALPHA_KE = {k:i for i, k in enumerate(np.unique(np.array(COORD_all.keys())[:,9]))}


param_space = np.zeros((len(R), len(T), len(I_T), len(KF), len(D1),
                        len(D6), len(D8), len(D10), len(ALPHA_KE))) + np.inf

for k, v in COORD_all.iteritems():
    param_space[R[k[0]], T[k[1]], I_T[k[3]],
                KF[k[4]], D1[k[5]], D6[k[6]],
                D8[k[7]], D10[k[8]], ALPHA_KE[k[9]]] = 4*v[0] + 2*v[1] + v[2]
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
min_val = 24
max_val = 60

output_folder_PS = output_folder + 'Param-sweep/'
if not os.path.exists(output_folder_PS):
    os.mkdir(output_folder_PS)

## Parameterization of the model
According to the parameter sweep and to the scoring of the different parametrizations as described before the best parametrization is picked

In [ ]:
COORD_all_sum = {k:np.sum(np.array(v) * np.array([4,2,0])) for k, v in COORD_all.iteritems()}
min_ = np.min(COORD_all_sum.values())
keys = COORD_all_sum.keys()
values = [keys[i] for i, v in enumerate(COORD_all_sum.values()) if v == min_]
results = [COORD_all[keys[i]] for i, v in enumerate(COORD_all_sum.values()) if v == min_]
ratio_th, low_th, hi_th, integration_time, gamma, kfLt_val, delta1, delta6, delta8, delta10 = values[0]
delta = [delta1, delta6, delta8, delta10]
# kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}

## Display of the optimal parameters

In [ ]:
df = {}
for rho_tmp, tau_tmp, tau2_tmp, IT_tmp, kfLt_tmp, delta1_tmp, delta6_tmp, delta8_tmp, delta10_tmp, aRke_tmp in values:
    df.setdefault(r'$\rho$', []).append(rho_tmp)
    df.setdefault(r'$\tau$', []).append(tau_tmp)
    df.setdefault('Integration time', []).append(IT_tmp)
    df.setdefault(r'$k_f.L^T$', []).append(kfLt_tmp)
    df.setdefault(r'$\delta_1$', []).append(delta1_tmp)
    df.setdefault(r'$\delta_{6,8}$', []).append(delta6_tmp)
    df.setdefault(r'$\delta_8$', []).append(delta8_tmp)
    df.setdefault(r'$\delta_{10}$', []).append(delta10_tmp)
    df.setdefault(r'$\alpha_R.k_e$', []).append(aRke_tmp)

for CP, OP, MP in results:
    df.setdefault('cell/ligand missed', []).append(CP)
    df.setdefault('Over predictions', []).append(OP)
    df.setdefault('Missed predictions', []).append(MP)
    
tmp = pd.DataFrame(df)
tmp = tmp[tmp.columns[[5, 6, 8, 7, 0, 1, 4, 3, 11, 10, 9]]]
display(tmp)

## Model run and figure generation

In [ ]:
reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
#### Here you can manually change the different parameters.
ratio_th, low_th, hi_th, integration_time, kfLt_val, delta1, delta6, delta8, delta10, aRke = values[1]
delta6, delta8 = 0.05, 0.05
delta = [delta1, delta6, delta8, delta10]
# ratio_th = 1.6
## THESE ARE THE COMMENTED EXAMPLES
# delta = 1e11 ### If this is uncommented it will set the delta to 1e11 for the next run
# kfLt_val = 1e-6 ### If this is uncommented it will set the kfLt to 1e-6 for the next run


kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}
begin = 2
end = 15
model = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1., ERK_delta = True)

model.run()
print model.format_output()

In [ ]:
output_folder = 'Model-Output-'+date.today().isoformat()+'/'

produce_basic_model_results(output_folder, model, values, results, 
                            cells_to_find_induction, cells_in_grey, 'WT Model')

test_model_no_delta = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta, sim_ratio_th=.10,
                              z_c_end=z_c_end, begin=begin, end=end, basic_euler = False,
                              size_increase = 1., ERK_delta = False)

test_model_no_delta.run()
test_model_no_delta.print_E(output_folder + 'E_values_all.csv')

## Parameter sweep study
From the parameter sweep results we can study how the model behave according to the different parameters.

The score is computed as described in the paper: 2 x missed cell/ligand interactions + missed specifications + over predictions

## $\tau$ vs $\rho$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (2, 3, 4, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(R.values())
ax.set_yticklabels(R.keys())
ax.set_xticks(T.values())
ax.set_xticklabels(T.keys(), rotation = 'vertical')
ax.set_xlabel(r'$\tau $', fontsize = 30)
ax.set_ylabel(r'$\rho $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'tau_vs_rho.pdf')

## Integration time vs $\rho$

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (1, 3, 4, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(R.values())
ax.set_yticklabels(R.keys())
ax.set_xticks(I_T.values())
ax.set_xticklabels(I_T.keys(), rotation = 'vertical')
ax.set_xlabel('Integration time [s]', fontsize = 30)
ax.set_ylabel(r'$\rho $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'Integration_time_vs_rho.pdf')

## Integration Time vs $k_f.L_t$

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 4, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(I_T.values())
ax.set_yticklabels(I_T.keys())
ax.set_xticks(KF.values())
ax.set_xticklabels(['%.1e'%v for v in KF.keys()], rotation='vertical')
ax.set_xlabel(r'$k_f.L_T$', fontsize = 30)
ax.set_ylabel('Integration Time [s]', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='horizontal')
fig.tight_layout()
plt.savefig(output_folder_PS+'Integration_time_vs_kflt.pdf')

## $k_f.L_t$ vs $\rho$

In [ ]:
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (1, 2, 4, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(R.values())
ax.set_yticklabels(R.keys())
ax.set_xticks(KF.values())
ax.set_xticklabels(['%.1e'%v for v in KF.keys()], rotation='vertical')
ax.set_xlabel(r'$k_f.L_T$', fontsize = 30)
ax.set_ylabel(r'$\rho $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='horizontal')
fig.tight_layout()
plt.savefig(output_folder_PS+'kfLt_vs_rho.pdf')

## $\delta_1$ vs $\rho$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (1, 2, 3, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(R.values())
ax.set_yticklabels(R.keys())
ax.set_xticks(D1.values())
ax.set_xticklabels(['%.1e'%v for v in D1.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_1$', fontsize = 30)
ax.set_ylabel(r'$\rho $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta_vs_rho.pdf')

## $\delta_1$ vs $\tau$

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 2, 3, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(T.values())
ax.set_yticklabels(T.keys())
ax.set_xticks(D1.values())
ax.set_xticklabels(['%.1e'%v for v in D1.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_1$', fontsize = 30)
ax.set_ylabel(r'$\tau $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta_vs_tau.pdf')

## $\delta_1$ vs $k_f.L_t$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(KF.values())
ax.set_yticklabels(['%.1e'%v for v in KF.keys()])
ax.set_xticks(D1.values())
ax.set_xticklabels(['%.1e'%v for v in D1.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_1$', fontsize = 30)
ax.set_ylabel(r'$k_f L_t $', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta_vs_kfLt.pdf')

## $\delta_1$ vs Integration time

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 3, 5, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(I_T.values())
ax.set_yticklabels(I_T.keys())
ax.set_xticks(D1.values())
ax.set_xticklabels(['%.1e'%v for v in D1.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_1$', fontsize = 30)
ax.set_ylabel('Integration Time', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta_vs_integration_time.pdf')

## $\delta_1$ vs $\delta_{6,8}$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 3, 6, 7, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(D1.values())
ax.set_yticklabels(['%.1e'%v for v in D1.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\delta_1$', fontsize = 30)
ax.set_xticks(D6.values())
ax.set_xticklabels(['%.1e'%v for v in D6.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_{6, 8}$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta1_vs_delta6.pdf')

## $\delta_1$ VS $\delta_{10}$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 3, 5, 6, 8)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(D1.values())
ax.set_yticklabels(['%.1e'%v for v in D1.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\delta_{1}$', fontsize = 30)
ax.set_xticks(D10.values())
ax.set_xticklabels(['%.1e'%v for v in D10.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\delta_{10}$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta1_vs_delta10.pdf')

## $\rho$ vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (1, 2, 3, 4, 5, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(R.values())
ax.set_yticklabels(['%.1f'%v for v in R.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\rho$', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'rho_vs_aRke.pdf')

## $\tau$ vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 2, 3, 4, 5, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(T.values())
ax.set_yticklabels(['%d'%v for v in T.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\tau$', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'tau_vs_aRke.pdf')

## Integration Time vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 3, 4, 5, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(I_T.values())
ax.set_yticklabels(['%d'%v for v in I_T.keys()], rotation = 'horizontal')
ax.set_ylabel(r'Integration Time [s]', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'Integrationtime_vs_aRke.pdf')

## $\alpha_R.k_e$ vs $kf.L_T$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 4, 5, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(KF.values())
ax.set_yticklabels(['%.1e'%v for v in KF.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$kf.L_T$', fontsize = 22)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 22)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'kfLt_vs_aRke.pdf')

## $\delta_1$ vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 3, 5, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(D1.values())
ax.set_yticklabels(['%.1e'%v for v in D1.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\delta_1$', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta1_vs_aRke.pdf')

## $\delta_{6,8}$ vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 3, 4, 6, 7)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(D6.values())
ax.set_yticklabels(['%.1e'%v for v in D6.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\delta_{6,8}$', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta6_vs_aRke.pdf')

## $\delta_{10}$ vs $\alpha_R.k_e$

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
img = ax.imshow(param_space.min(axis = (0, 1, 2, 3, 4, 5, 6)),
                interpolation = 'nearest', origin = 'lower', vmin=min_val, vmax = max_val)
ax.set_yticks(D10.values())
ax.set_yticklabels(['%.1e'%v for v in D10.keys()], rotation = 'horizontal')
ax.set_ylabel(r'$\delta_{10}$', fontsize = 30)
ax.set_xticks(ALPHA_KE.values())
ax.set_xticklabels(['%.1e'%v for v in ALPHA_KE.keys()], rotation = 'vertical')
ax.set_xlabel(r'$\alpha_R.k_e$', fontsize = 30)
fig.colorbar(img, ax =ax, orientation='vertical')
fig.tight_layout()
plt.savefig(output_folder_PS+'delta10_vs_aRke.pdf')

## Plots of the distribution of the $E^\star$

In [ ]:
test_model = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = 600, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1., ERK_delta = True)
distribution_per_pw = {}
for di in test_model.gene_repartition.itervalues():
    for dii in di.itervalues():
        for pw, v in dii.iteritems():
            if v != 0:
                distribution_per_pw.setdefault(pw[0], []).append(v)

output_folder_E_star = output_folder + 'E_cumm_hist/'
if not os.path.exists(output_folder_E_star):
    os.mkdir(output_folder_E_star)

In [ ]:
for pw, d in distribution_per_pw.iteritems():
    if not 'ERK' in pw and not 'tolloid' in pw and not '-' in pw:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111)
        tmp = ax.hist(d, normed=True, cumulative=True,
                      bins = 10000, #range=(0, 2.5*10**9),
                      histtype='step', lw=3, alpha=.6, color = 'k')
        y_v = np.min(tmp[0][tmp[0]>=low_th/100.])
        x_v = np.min(tmp[1][1:][tmp[0]>=low_th/100.])
        ax.plot([0, x_v], [y_v, y_v], 'r-', lw=4, alpha = .5)
        ax.plot([x_v, x_v], [0, y_v], 'r-', lw=4, alpha = .5)
        ax.set_title(pw[:-1], fontsize = 25)
        ax.set_ylim(0, 1)
        ax.set_xlim(0, np.max(d))
        ax.set_yticks(sorted([0, .5, 1.] + [y_v]))
        ax.set_xticks(sorted([np.max(d)/2, np.max(d)] + [x_v]))
        ax.set_yticklabels([0, r'$\tau$', 50, 100], fontsize=25)
        ax.set_xticklabels((('$\overline{E}^*_{min}=$\n$%.0e$ '%x_v\
                             +('%.0e '*2)%tuple(sorted([np.max(d)/2, np.max(d)])))).split(' '))
        ax.tick_params(axis='y', which='both', labelsize=20)
        ax.tick_params(axis='x', which='both', labelsize=20)
        # ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0), labelsize=20)
        ax.set_xlabel('$\overline{E}^*($' + pw[:-1] + '$)$', fontsize=25)
        ax.set_ylabel('% of cells with effector values $< E^*$', fontsize=25)
        fig.tight_layout()
        plt.savefig(output_folder_E_star + 'Estar_distrib' + pw[:-1] + '.pdf')

In [ ]:
# pw = 'ERK+'
d = distribution_per_pw['ERK+']
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
tmp = ax.hist(d, normed=True, cumulative=True,
              bins = 10000, range=(0, 1.8),
              histtype='step', lw=3, alpha=.6, color = 'k')
y_v = np.min(tmp[0][tmp[0]>=low_th/100.])
x_v = np.min(tmp[1][1:][tmp[0]>=low_th/100.])
ax.plot([0, x_v], [y_v, y_v], 'r-', lw=4, alpha = .5)
ax.plot([x_v, x_v], [0, y_v], 'r-', lw=4, alpha = .5) 
ax.set_title('Ras', fontsize = 25)
ax.set_ylim(0, 1)
ax.set_xlim(np.min(tmp[1]), tmp[1][np.min(np.where(tmp[0]>.9999))])
ax.set_yticks(sorted([0, .5, 1.] + [y_v]))
d = tmp[1][:np.min(np.where(tmp[0]>.9999))]
ax.set_xticks(sorted([np.max(d)/2, np.max(d)] + [x_v]))
ax.set_xticklabels(('$E^\star_{min}$ '+('%.0e '*2)%tuple(sorted([np.max(d)/2, np.max(d)]))).split())
ax.set_yticklabels([0, r'$\tau$', 50, 100], fontsize=25)
# ax.tick_params(axis='y', which='major', labelsize=20)
ax.tick_params(axis='x', which='major', labelsize=20)
ax.set_xlabel(r'$RasGTP$', fontsize=25)
ax.set_ylabel('% of cells with effector values $< RasGTP$', fontsize=25)
fig.tight_layout()
plt.savefig(output_folder_E_star + 'Estar_distrib' + pw[:-1] + '.pdf')

## Combined plots of the different model included in the main paper

In [ ]:
reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
begin = 2
end = 15
model = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression, gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = False, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              basic_euler = False, size_increase = 1., ERK_delta = True)

model.run()

gene_expression_no_Eph = {}
for k, v in gene_expression.iteritems():
    if not 'ERK-' in k:
        gene_expression_no_Eph[k] = v
model_no_eph = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression_no_Eph, gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = False, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              basic_euler = False, size_increase = 1., ERK_delta = True,
                              max_E_for_induction = model.max_E_for_induction,
                              min_E_for_uninduction = model.min_E_for_uninduction,
                              gene_cat_low = model.gene_cat_low)

model_no_eph.run()

(ratio_th_no_surf, low_th_no_surf,
 hi_th_no_surf, integration_time_no_surf,
 kfLt_val_no_surf, delta1_no_surf, delta6_no_surf,
 delta8_no_surf, delta10_no_surf, aRke_no_surf) = (2.2, 38, 100, 600, 0.0001, 10.0, 0.001, 
                                                   0.001, 0.10000000000000001, 9.9999999999999995e-07)

kfLt_no_surf = {k:kfLt_val_no_surf for k in gene_expression.iterkeys()}
delta_no_surf = [delta1_no_surf, delta6_no_surf, delta8_no_surf, delta10_no_surf]

model_no_surf = InteractionModel(lin_tree, surf_ex, couples, low_th_no_surf, hi_th_no_surf, ratio_th_no_surf,
                              kfLt_no_surf, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              delta = delta_no_surf,take_surf = False, alphaR_ke = aRke_no_surf,
                              integration_time = integration_time_no_surf, dt = 2, sim_ratio_th=.10,
                              rand = False, ratio_noise = 0, z_c_end=z_c_end, begin=2, end=15,
                              basic_euler = False, size_increase = 1., ERK_delta = True)

model_no_surf.run()

f = open('DATA/output_rnd_exp_pat.pkl')
data_rnd = pkl.load(f)
f.close()
avg = np.mean(data_rnd, axis = 1)
found = avg[5] - avg[4]
missed = avg[4]
avg[4] = found
avg[5] = missed
std = np.std(data_rnd, axis = 1)
found = std[5] - std[4]
missed = std[4]
std[4] = found
std[5] = missed

output_folder_main_fig = output_folder + 'Main-figures/'
if not os.path.exists(output_folder_main_fig):
    os.mkdir(output_folder_main_fig)
if not os.path.exists(output_folder_main_fig + 'noise/'):
    os.mkdir(output_folder_main_fig + 'noise/')

In [ ]:
def plot_model_results_hor(model, init_pos, nb, ax, col, space = 2, values = None, error_bars = None, hatch = ''):
    if values is None:
        a, b, c, d, e, f = model.retrieve_the_cat()
    else:
        a, b, c, d, e, f = values
    p1 = init_pos + .2
    p2 = init_pos + .2
    ax.barh(p1, a/float(a+b)*100-1, left = 1, height = 1.6, color = cm.Paired(.4), lw = 2)
    ax.barh(p2, -e/float(e+f)*100+1, left = -1, height = 1.6, color = cm.Paired(0), lw = 2)
    if not error_bars is None:
        ax.errorbar([a/float(a+b)*100, -e/float(e+f)*100], [p1, p2],
                    xerr=[error_bars[0], error_bars[4]], fmt='none', ecolor='k', linewidth=3, mew=3, alpha = .7)


import matplotlib
import mpl_toolkits.axisartist as axisartist
font = {'family' : 'normal',
          'weight' : 'normal',
          'size'   : 32}
matplotlib.rc('font', **font)

fig = plt.figure(figsize=(11, 8.5))
ax = axisartist.Subplot(fig, 111)
# ax = fig.add_subplot(111)
fig.add_subplot(ax)
plot_model_results_hor(model, 7.8, 4, ax, col = 'b')
plot_model_results_hor(model_no_eph, 1.8, 4, ax, col = 'm')
plot_model_results_hor(model_no_surf, 3.8, 4, ax, col = 'y')
plot_model_results_hor(None, 5.8, 4, ax, col = 'c', error_bars=std, values=avg)
ax.bar(0, 0, color = cm.Paired(0), label = '% of known induction events predicted')
ax.bar(0, 0, color = cm.Paired(.4), label = '% of over prediction')
ax.legend(loc = 'upper left', fontsize = 25)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_ylim(1, 11)
ax.set_yticks([2, 4, 6, 8])
ax.set_yticklabels(["Eph -", "Constant #receptor\n@cell-cell interface", "Gene expression\nrandomization", "WT"])
ax.set_xticks(range(-100, 101, 20))
ax.set_xticklabels(['%d'%v for v in range(100, 0, -20) + range(0, 101, 20)])
ax.set_xlim(-115, 40)
ax.set_xlabel('Percentage', fontsize = 40)
ax.axis['left'].major_ticks.set_tick_out(True)
ax.axis['left'].invert_ticklabel_direction()
fig.tight_layout()
plt.savefig(output_folder_main_fig + 'Global-output.pdf')
plt.sty

In [ ]:
f = open('DATA/output_rand_surf.pkl')
FP_noise_mem_T, FN_noise_mem_T, MISSED_L_noise_mem_T, C_P_M, IND_noise_mem_T = pkl.load(f)
f.close()

nb_runs = 96.
X_mem = list(np.arange(0, 31, 2.))
fig = plt.figure(figsize=(11, 6.8))
ax = fig.add_subplot(111)
plot_box_like_distrib(X_mem, MISSED_L_noise_mem_T, ax, 0, 'o',
                      '% of known induction\nevents predicted', max_val=31, div = 14., reverse = True)[0]
ax.set_xlabel('% of noise in cell-cell\ncontact area (average)', fontsize = 35)
# ax.legend(loc = 'lower left', fontsize = 30)
ax.tick_params(axis='both', which='major', labelsize=30)
ax.set_ylabel('% of known induction\nevents predicted', fontsize = 35)
ax.set_xlim(-1, 24)
ax.set_ylim(60, 100)
fig.tight_layout()
fig.savefig(output_folder_main_fig + 'random_missed_lig.pdf')

from natsort import natsorted
# viridis = cm.viridis
t_c = np.max(X_mem)
for amp, C_P_M_tmp in C_P_M.iteritems():
    if amp == 20:
        fig = plt.figure(figsize=(11, 6.8))
        ax = fig.add_subplot(111)
        keys_s = [k for k in natsorted(C_P_M_tmp)]# if k != ('a6.4', 'Nodal+', 'Nodal+', 'Stage 8')]
        v_sorted = np.array([C_P_M_tmp[k] for k in keys_s])
        # ax.plot(100*v_sorted/200., 'o', color = 'k')#viridis(amp/t_c))
        ax.bar(np.arange(-0.5, 13.5, 1), 100*v_sorted/nb_runs, color = cm.Paired(0))
        ax.set_ylim(0, 70)
        ax.set_xlim(-1, 13)
        ax.set_yticks(range(0, 71, 20))
        ax.set_xticks(np.arange(-.5, 13, 1))
        ax.set_xticklabels([k[0] + ', ' + k[1][:-1] + ', ' + 'S' + k[3].split(' ')[-1] for k in keys_s],
                           rotation = 'vertical', fontsize = 30)

        ax.tick_params(axis = 'y', labelsize = 30)
        ax.set_ylabel('% of time\nmissed', fontsize = 35)
        ax.set_title('Average noise amplitude of 20%', fontsize = 35)
        fig.tight_layout()
        fig.savefig(output_folder_main_fig + 'noise/Number_of_missed_pairs_amp_%d.pdf'%amp)
        if amp != 20:
            plt.clf()


In [ ]:
f = open('DATA/size_scaling5.1.pkl')
FP_size, FN_size, MISSED_L_size, COORD_size = pkl.load(f)
f.close()
from matplotlib import ticker
si_vals = (np.log10(.25), np.log10(4), 10)
X = sorted(set([1.] + list(np.logspace(*si_vals))))

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(X, 100*FP_size/56., 'o-', lw=2, c = cm.Paired(.4),
        label = '% of over predictions', ms = 8)
# ax.plot(X, 100*FN_size/18., 'o--', lw=2, alpha = .5, label = 'Missed inductions')
ax.plot(X, 100-100*MISSED_L_size/14., 'o-', lw=2, c = cm.Paired(0), 
        label = '% of known induction events predicted', ms = 8)
ax.legend(loc='center left', fontsize = 20)
ax.set_xlabel('Scaling factor cell-cell contact area', fontsize = 25)
ax.set_ylabel('percentage', fontsize = 25)
ax.set_xscale('log')
ax.tick_params(axis = 'both', labelsize = 20)
ax.set_xlim((.23,4.5))
# ax.set_ylim((0, 30))
ax.set_xticks([.25, .5, 1, 2, 4])
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
ax.set_ylim(0, 105)
fig.tight_layout()
fig.savefig(output_folder_main_fig + 'size_variation.pdf')


In [ ]:
f = open('DATA/size_scaling_const_teta5.1.pkl')
FP_size, FN_size, MISSED_L_size, COORD_size = pkl.load(f)
f.close()
from matplotlib import ticker
si_vals = (np.log10(.25), np.log10(4), 10)
X = sorted(set([1.] + list(np.logspace(*si_vals))))

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(X, [COORD_size[k][4] for k in X], 'o-', lw=2, c = cm.Paired(.79),
        label = '', ms = 8)
# ax.legend(loc='center left', fontsize = 20)
ax.set_xlabel('Scaling factor cell-cell contact area', fontsize = 25)
ax.set_ylabel(r'Optimal $k_f.L_T$', fontsize = 25)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis = 'both', labelsize = 20)
ax.set_xlim((.23,4.5))
ax.set_xticks([.25, .5, 1, 2, 4])
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
yrange_vals = np.logspace(-1, -10, 19)[4:8]
ax.set_ylim((yrange_vals[-1]/2, yrange_vals[0]*2))
ax.set_yticks(yrange_vals)
ax.set_yticklabels((('%.1e '*4)%tuple(yrange_vals)).split(' ')[:-1])
fig.tight_layout()
fig.savefig(output_folder_main_fig + 'kfLt_for_teta_fixed.pdf')

# Different variations of the model:
Ephrin Knock out, Constant number of receptors, randomization of the surface of contact, randomization of the expression, scaling of the surface of contact.

In [ ]:
output_folder_no_eph = 'Model-Output-Mutants-'+date.today().isoformat()+'/eph-/'
output_folder_no_surf = 'Model-Output-Mutants-'+date.today().isoformat()+'/const-rec/'
output_folder_rnd_surf = 'Model-Output-Mutants-'+date.today().isoformat()+'/rnd_surf/'
output_folder_rnd_exp = 'Model-Output-Mutants-'+date.today().isoformat()+'/rnd_exp/'
output_folder_surf_inc = 'Model-Output-Mutants-'+date.today().isoformat()+'/surf_inc/'

produce_basic_model_results(output_folder_no_eph, model_no_eph, [], [], 
                            cells_to_find_induction, cells_in_grey, 'Eph -')
produce_basic_model_results(output_folder_no_surf, model_no_surf, [], [],
                            cells_to_find_induction, cells_in_grey, 'Constant #receptor @cell-cell interface')

model_rnd_surf = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression, gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 20., alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              basic_euler = False, size_increase = 1., ERK_delta = True,
                                 min_E_for_uninduction = model.min_E_for_uninduction,
                                 max_E_for_induction = model.max_E_for_induction)
model_rnd_surf.run()
produce_basic_model_results(output_folder_rnd_surf, model_rnd_surf, [], [],
                            cells_to_find_induction, cells_in_grey, '20% noise in cell-cell contact area')

gene_expression_rand = {k:set([randomize_name(n) for n in v]) for k, v in gene_expression.iteritems()}
model_rnd_exp = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression_rand, gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 20., alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              basic_euler = False, size_increase = 1., ERK_delta = True)
model_rnd_exp.run()
produce_basic_model_results(output_folder_rnd_exp, model_rnd_exp, [], [],
                            cells_to_find_induction, cells_in_grey, 'Randomized gene expression')

In [ ]:
f = open('DATA/size_scaling5.1.pkl')
FP_size, FN_size, MISSED_L_size, COORD_size = pkl.load(f)
f.close()
(ratio_th_size, low_th_size, hi_th_size,
 integration_time_size, kfLt_val_size, delta1_size,
 delta6_size, delta8_size, delta10_size, aRke_size) = COORD_size[4.]

kfLt_size = {k:kfLt_val_size for k in kfLt.keys()}
delta_size = [delta1_size, delta6_size, delta8_size, delta10_size]

model_surf_inc = InteractionModel(lin_tree, surf_ex, couples, low_th_size, hi_th_size, ratio_th_size,
                              kfLt_size, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression, gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time_size, dt = dt, delta = delta_size,
                              sim_ratio_th=.10, rand = False, ratio_noise = 20., alphaR_ke = aRke_size,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              basic_euler = False, size_increase = 4., ERK_delta = True)
model_surf_inc.run()

produce_basic_model_results(output_folder_surf_inc, model_surf_inc, [], [],
                            cells_to_find_induction, cells_in_grey, r'$4\times$ scaling')



# Model version with no antagonist expression

In [ ]:
reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
#### Here you can manually change the different parameters.
ratio_th, low_th, hi_th, integration_time, kfLt_val, delta1, delta6, delta8, delta10, aRke = values[2]
delta6, delta8 = 0.05, 0.05
delta = [delta1, delta6, delta8, delta10]
# ratio_th = 1.6
## THESE ARE THE COMMENTED EXAMPLES
# delta = 1e11 ### If this is uncommented it will set the delta to 1e11 for the next run
# kfLt_val = 1e-6 ### If this is uncommented it will set the kfLt to 1e-6 for the next run

gene_expression_no_minus = {}
for k, v in gene_expression.iteritems():
    if not '-' in k[0]:
        gene_expression_no_minus[k] = v
        
kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}
begin = 2
end = 15
model_no_minus = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression_no_minus,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end = time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1., ERK_delta = False)
model_no_minus.run()

# Model variation with no ratio between ERK and Eph applyied
It is necessary to access these two values independently

In [ ]:
reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
#### Here you can manually change the different parameters.
ratio_th, low_th, hi_th, integration_time, kfLt_val, delta1, delta6, delta8, delta10, aRke = values[2]
delta6, delta8 = 0.05, 0.05
delta = [delta1, delta6, delta8, delta10]
# ratio_th = 1.6
## THESE ARE THE COMMENTED EXAMPLES
# delta = 1e11 ### If this is uncommented it will set the delta to 1e11 for the next run
# kfLt_val = 1e-6 ### If this is uncommented it will set the kfLt to 1e-6 for the next run

kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}
begin = 2
end = 15
model_no_delta = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end = time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1., ERK_delta = False)
model_no_delta.run()
# print model_tmp.format_output()


# Model variation with only binary contacts (no surface)

In [ ]:
reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
#### Here you can manually change the different parameters.
ratio_th, low_th, hi_th, integration_time, kfLt_val, delta1, delta6, delta8, delta10, aRke = values[2]
delta6, delta8 = 0.05, 0.05
delta = [delta1, delta6, delta8, delta10]
# ratio_th = 1.6
## THESE ARE THE COMMENTED EXAMPLES
# delta = 1e11 ### If this is uncommented it will set the delta to 1e11 for the next run
# kfLt_val = 1e-6 ### If this is uncommented it will set the kfLt to 1e-6 for the next run

kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}
begin = 2
end = 15
model = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end = time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1., ERK_delta = True)
model.run()

(ratio_th_no_surf, low_th_no_surf,
 hi_th_no_surf, integration_time_no_surf,
 kfLt_val_no_surf, delta1_no_surf, delta6_no_surf,
 delta8_no_surf, delta10_no_surf, aRke_no_surf) = ((2.0, 40, 100, 720,
                                      0.00031622776601683794,
                                      10.0, 0.001, 0.001,
                                      10.0, 9.9999999999999995e-07))

kfLt_no_surf = {k:kfLt_val_no_surf for k in gene_expression.iterkeys()}
delta_no_surf = [delta1_no_surf, delta6_no_surf, delta8_no_surf, delta10_no_surf]

model_no_surf = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                                 kfLt, cells_to_find_induction, fates2, names, fates3_names,
                                 sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                                 gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                                 ground_truth, ground_truth_pw, time_end=time_end, grey_zone = cells_in_grey,
                                 delta = delta,take_surf = False, alphaR_ke = aRke,
                                 integration_time = integration_time, dt = 2, sim_ratio_th=.10,
                                 rand = False, ratio_noise = 0, z_c_end=z_c_end, begin=2, end=15,
                                 basic_euler = False, size_increase = 1., ERK_delta = True,
                                 max_E_for_induction = model.max_E_for_induction,
                                 min_E_for_uninduction = model.min_E_for_uninduction)

model_no_surf.run()

# Model variation with +/- 20% signal

In [ ]:

reload(ClassModel)
InteractionModel = ClassModel.InteractionModel
#### Here you can manually change the different parameters.
ratio_th, low_th, hi_th, integration_time, kfLt_val, delta1, delta6, delta8, delta10, aRke = values[2]
delta6, delta8 = 0.05, 0.05
delta = [delta1, delta6, delta8, delta10]
# ratio_th = 1.6
## THESE ARE THE COMMENTED EXAMPLES
# delta = 1e11 ### If this is uncommented it will set the delta to 1e11 for the next run
# kfLt_val = 1e-6 ### If this is uncommented it will set the kfLt to 1e-6 for the next run

perc = .5
kfLt = {k:kfLt_val for k in gene_expression.iterkeys()}
begin = 2
end = 15
model_20P = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end = time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1, ERK_delta = True,
                                 min_E_for_uninduction = model.min_E_for_uninduction,
                                 max_E_for_induction = model.max_E_for_induction, signal_increase = 1 + perc)
model_20P.run()

model_20M = InteractionModel(lin_tree, surf_ex, couples, low_th, hi_th, ratio_th,
                              kfLt, cells_to_find_induction, fates2, names, fates3_names,
                              sim_nv, vol, surfaces, gene_expression,gene_affect_neighb, 
                              gene_affect_self, inv_lin_tree, pathways, stages,gene_effect,
                              ground_truth, ground_truth_pw, time_end = time_end, grey_zone = cells_in_grey,
                              integration_time = integration_time, dt = dt, delta = delta,
                              sim_ratio_th=.10, rand = True, ratio_noise = 0, alphaR_ke = aRke,
                              z_c_end=z_c_end, begin=begin, end=end, known_interactions = known_interactions,
                              size_increase = 1, ERK_delta = True,
                                 min_E_for_uninduction = model.min_E_for_uninduction,
                                 max_E_for_induction = model.max_E_for_induction, signal_increase = 1 - perc)
model_20M.run()

In [ ]:
stage_selected = 11
test = lambda st: st == 11
test = lambda st: True

c_see_stage = {short_name(names[ci]):[] for c, trash in couples for ci in lin_tree[c]}
for c, see in model_no_delta.c_see.iteritems():
    st = max([int(k[-1].split(' ')[-1]) for k in see])
    if test(st):
        c_see_stage[c] = [k for k in see if not 'ERK-' in k[0]]# and int(k[-1].split(' ')[-1]) == st]
plus_minus_see = c_see_stage
plus_minus = [len(k) for k in c_see_stage.itervalues()]

c_see_stage = {short_name(names[ci]):[] for c, trash in couples for ci in lin_tree[c]}
for c, see in model.c_induced.iteritems():
    st = max([int(k[0][-1].split(' ')[-1]) for k in see])
    if test(st) and not '6.' in c:
        c_see_stage[c] = [k[0] for k in see]# if int(k[0][-1].split(' ')[-1]) == st and k[1] == 1]
induced = [len(k) for k in c_see_stage.itervalues()]
induced_see = c_see_stage

bins = 11
range_ = (0, 11)
ylim = (0, 100)

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
tmp = ax.hist(plus_minus, bins = bins, range = range_, align = 'left',
        label = 'total #pathways seen: %d'%(np.sum(plus_minus)))
ax.set_xlabel('#inducers', size = 35)
ax.set_ylabel('#cells', size = 35)
ax.legend(fontsize = 25)
ax.set_xticks(range(0, 11, 2))
ax.tick_params(axis = 'both', labelsize = 30)
ax.set_ylim(ylim)

fig.tight_layout()
fig.savefig(output_folder + '5C.pdf')

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
ax.hist(induced, bins = bins, range = range_, align = 'left',
        label = '#inducers: %d\n#induced cells: %d'%(np.sum(induced),
                                                     len([i for i in induced if i > 0])))
ax.set_xlabel('#inducers', size = 35)
ax.set_ylabel('#cells', size = 35)
ax.set_xticks(range(0, 11, 2))
ax.legend(fontsize = 25)
ax.tick_params(axis = 'both', labelsize = 30)
ax.set_ylim(ylim)
fig.tight_layout()
fig.savefig(output_folder + '5D.pdf')

In [ ]:
X = [5, 10, 20, 30, 40, 50]
v112 = np.array([16, 18, 30, 44, 60, 76])
v64 = np.array([12, 14, 22, 28, 34, 38])

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

ax.plot(X, v112, 'o-', label = '112-cell stage', ms = 10)
ax.plot(X,  v64, 'o-', label = '64-cell stage', ms = 10)
ax.plot([], [], 'ko--', label = 'Normalized to the\nnumber of cells', ms = 10)
ax.set_xlabel('increase/decrease [% of initial signal]', fontsize = 30)
ax.set_ylabel('#signaling states changed', fontsize = 30)
ax.tick_params(axis = 'both', labelsize = 25)
ax2 = ax.twinx()

ax2.plot(X, v112/112., 'o--', label = 'Normalized', ms = 10)
ax2.plot(X,  v64/ 64., 'o--', label = 'Normalized', ms = 10)
ax2.tick_params(axis = 'both', labelsize = 25)
ax2.set_ylabel('% of signaling states changed', fontsize = 30)
ax.legend(fontsize = 25)

fig.tight_layout()
plt.savefig(output_folder + 'change-in-signal.pdf')